# SNMP - Simple Network Management Protocol

## Install snmpd tools, agent and mibs

In [1]:
%%bash

sudo apt update
sudo apt install -y snmp snmpd snmp-mibs-downloader

Hit:1 http://archive.ubuntu.com/ubuntu focal InRelease
Get:2 http://archive.ubuntu.com/ubuntu focal-updates InRelease [128 kB]
Get:3 https://packages.microsoft.com/repos/microsoft-ubuntu-focal-prod focal InRelease [3632 B]
Hit:4 https://dl.yarnpkg.com/debian stable InRelease
Hit:5 http://archive.ubuntu.com/ubuntu focal-backports InRelease
Hit:6 https://repo.anaconda.com/pkgs/misc/debrepo/conda stable InRelease
Get:7 http://security.ubuntu.com/ubuntu focal-security InRelease [128 kB]
Get:8 https://packages.microsoft.com/repos/microsoft-ubuntu-focal-prod focal/main amd64 Packages [297 kB]
Get:10 http://archive.ubuntu.com/ubuntu focal-updates/restricted amd64 Packages [3882 kB]
Get:11 http://archive.ubuntu.com/ubuntu focal-updates/universe amd64 Packages [1521 kB]
Get:12 http://archive.ubuntu.com/ubuntu focal-updates/main amd64 Packages [4227 kB]
Hit:9 https://packagecloud.io/github/git-lfs/ubuntu focal InRelease
Get:13 http://security.ubuntu.com/ubuntu focal-security/main amd64 Packages 

Reading package lists...
Building dependency tree...
Reading state information...
snmp-mibs-downloader is already the newest version (1.2).
snmp is already the newest version (5.8+dfsg-2ubuntu2.9).
snmpd is already the newest version (5.8+dfsg-2ubuntu2.9).
0 upgraded, 0 newly installed, 0 to remove and 31 not upgraded.


## Configure agent configuration file (/etc/snmp/snmpd.conf) and snmp tools configuration file (/etc/snmp/snmp.conf)

In [2]:
%%bash

sudo mv /etc/snmp/snmpd.conf /etc/snmp/snmpd.conf.old

sudo tee /etc/snmp/snmpd.conf > /dev/null <<EOF
rocommunity public
rwcommunity private
EOF

sudo sed -i '/mibs :/s/^/#/' /etc/snmp/snmp.conf

## Start snmpd daemon (agent)

In [3]:
%%bash

sudo service snmpd restart
sudo service snmpd status

 * snmpd is running


## List all downloaded mibs

In [4]:
%%bash

ls -R /usr/share/snmp/mibs

/usr/share/snmp/mibs:
CarroAutonomo.txt
GNOME-SMI.txt
LM-SENSORS-MIB.txt
NET-SNMP-AGENT-MIB.txt
NET-SNMP-EXAMPLES-MIB.txt
NET-SNMP-EXTEND-MIB.txt
NET-SNMP-MIB.txt
NET-SNMP-MONITOR-MIB.txt
NET-SNMP-PASS-MIB.txt
NET-SNMP-PERIODIC-NOTIFY-MIB.txt
NET-SNMP-SYSTEM-MIB.txt
NET-SNMP-TC.txt
NET-SNMP-VACM-MIB.txt
UCD-DEMO-MIB.txt
UCD-DISKIO-MIB.txt
UCD-DLMOD-MIB.txt
UCD-IPFILTER-MIB.txt
UCD-IPFWACC-MIB.txt
UCD-SNMP-MIB-OLD.txt
UCD-SNMP-MIB.txt
iana
ietf


## Show CarroAutonomo object tree

In [16]:
%%bash

# Variables
MIB="/usr/share/snmp/mibs/CarroAutonomo.txt"

snmptranslate -m $MIB -Tp

Unlinked OID in CarroAutonomo-MIB: carroAutonomoMIB ::= { experimental 9999 }
Undefined identifier: experimental near line 9 of /usr/share/snmp/mibs/CarroAutonomo.txt


+--iso(1)
   |
   +--org(3)
      |
      +--dod(6)
         |
         +--internet(1)
            |
            +--directory(1)
            |
            +--mgmt(2)
            |  |
            |  +--mib-2(1)
            |     |
            |     +--transmission(10)
            |
            +--experimental(3)
            |  |
            |  +--carroAutonomoMIB(9999)
            |     +-- -R-- String    manufacturer(1)
            |     |        Textual Convention: DisplayString
            |     |        Size: 0..255
            |     +-- -R-- String    model(2)
            |     |        Textual Convention: DisplayString
            |     |        Size: 0..255
            |     +-- -R-- String    serialNumber(3)
            |     |        Textual Convention: DisplayString
            |     |        Size: 0..255
            |     +-- -R-- String    softwareVersion(4)
            |     |        Textual Convention: DisplayString
            |     |        Size: 0..255
            |    

## Show the description of a specific object

In [17]:
%%bash

# Variables
OBJECT="batteryLevel"
MIB="/usr/share/snmp/mibs/CarroAutonomo.txt"

# Command to extract text from search_string to } excluding the }
sed -n "/$OBJECT OBJECT-TYPE/,/}/p" $MIB

batteryLevel OBJECT-TYPE
    SYNTAX  Integer32 (0..100)
    MAX-ACCESS read-only
    STATUS  current
    DESCRIPTION
        "Current battery level of the autonomous car (percentage)."
    ::= { carroAutonomoMIB 10 }


# Example using pass directive

## snmpd.conf

In [5]:
%%bash

# Add pass directive to snmpd.conf
sudo tee /etc/snmp/snmpd.conf > /dev/null <<EOF
rocommunity public
rwcommunity private

pass .1.3.6.1.3.9999 /usr/bin/python3 /tmp/agent.py
EOF

# Restart agent
sudo service snmpd restart
sudo service snmpd status

 * snmpd is running


## Agent code

In [18]:
%%writefile /tmp/agent.py

#!/usr/bin/env python3

import sys
import os
import tempfile

# Definindo o arquivo de dados no diretório temporário do sistema
data_file = os.path.join(tempfile.gettempdir(), 'agent_data.txt')

# Variáveis iniciais
manufacturer = "Tesla"  # The manufacturer of the autonomous car.
model = "S"  # The model of the autonomous car.
serialNumber = "5YJ3E1EA7JF000000"  # The serial number of the autonomous car.
softwareVersion = "2023.20.11"  # The software version running on the autonomous car.
desiredSpeed = 100  # The desired speed set for the autonomous car.
currentSpeed = 0  # The current speed of the autonomous car.
engineStatus = 1  # off(1), on(2), maintenance(3)
distanceTraveled = 0  # Total distance traveled by the autonomous car.
fuelLevel = 100  # Current fuel level of the autonomous car (percentage).
batteryLevel = 30  # Current battery level of the autonomous car (percentage).
maintenanceMode = 0  # Indicates if the maintenance mode is enabled or disabled.

# Mapeamento de OIDs para seus sucessores
oid_map = {
    ".1.3.6.1.3.9999.1.0": ".1.3.6.1.3.9999.2.0",
    ".1.3.6.1.3.9999.2.0": ".1.3.6.1.3.9999.3.0",
    ".1.3.6.1.3.9999.3.0": ".1.3.6.1.3.9999.4.0",
    ".1.3.6.1.3.9999.4.0": ".1.3.6.1.3.9999.5.0",
    ".1.3.6.1.3.9999.5.0": ".1.3.6.1.3.9999.6.0",
    ".1.3.6.1.3.9999.6.0": ".1.3.6.1.3.9999.7.0",
    ".1.3.6.1.3.9999.7.0": ".1.3.6.1.3.9999.8.0",
    ".1.3.6.1.3.9999.8.0": ".1.3.6.1.3.9999.9.0",
    ".1.3.6.1.3.9999.9.0": ".1.3.6.1.3.9999.10.0",
    ".1.3.6.1.3.9999.10.0": ".1.3.6.1.3.9999.11.0",
}

# Funções de obtenção de valores
def get_manufacturer():
    return manufacturer

def get_model():
    return model

def get_serialNumber():
    return serialNumber

def get_softwareVersion():
    return softwareVersion

def get_desiredSpeed():
    return desiredSpeed

def get_currentSpeed():
    return currentSpeed

def get_engineStatus():
    return engineStatus

def get_distanceTraveled():
    return distanceTraveled

def get_fuelLevel():
    return fuelLevel

def get_batteryLevel():
    return batteryLevel

def get_maintenanceMode():
    return maintenanceMode

# Funções de alteração de valores
def set_desiredSpeed(value):
    global desiredSpeed
    desiredSpeed = int(value)
    save_values()
    return desiredSpeed

def set_engineStatus(value):
    global engineStatus
    engineStatus = int(value)
    save_values()
    return engineStatus

def set_maintenanceMode(value):
    global maintenanceMode
    maintenanceMode = int(value)
    save_values()
    return maintenanceMode

# Função para salvar os valores no arquivo
def save_values():
    with open(data_file, 'w') as file:
        file.write(f"{desiredSpeed},{engineStatus}")

# Função para carregar os valores do arquivo
def load_values():
    global desiredSpeed, engineStatus
    try:
        with open(data_file, 'r') as file:
            data = file.readline().strip()
            if data:
                desiredSpeed, engineStatus = map(int, data.split(','))
    except FileNotFoundError:
        pass  # Arquivo ainda não existe, valores padrão serão usados

# Função para responder a requisições GET
def get_response(oid):
    if oid == ".1.3.6.1.3.9999.1.0": # MANUFACTURER
        print(".1.3.6.1.3.9999.1.0")
        print("string")
        print(get_manufacturer())
    elif oid == ".1.3.6.1.3.9999.2.0": # MODEL
        print(".1.3.6.1.3.9999.2.0")
        print("string")
        print(get_model())
    elif oid == ".1.3.6.1.3.9999.3.0": # serialNumber
        print(".1.3.6.1.3.9999.3.0")
        print("string")
        print(get_serialNumber())
    elif oid == ".1.3.6.1.3.9999.4.0": # softwareVersion
        print(".1.3.6.1.3.9999.4.0")
        print("string")
        print(get_softwareVersion())
    elif oid == ".1.3.6.1.3.9999.5.0": # desiredSpeed
        print(".1.3.6.1.3.9999.5.0")
        print("integer")
        print(get_desiredSpeed())
    elif oid == ".1.3.6.1.3.9999.6.0": # currentSpeed
        print(".1.3.6.1.3.9999.6.0")
        print("integer")
        print(get_currentSpeed())
    elif oid == ".1.3.6.1.3.9999.7.0": # engineStatus
        print(".1.3.6.1.3.9999.7.0")
        print("integer")
        print(get_engineStatus())
    elif oid == ".1.3.6.1.3.9999.8.0": # distanceTraveled
        print(".1.3.6.1.3.9999.8.0")
        print("integer")
        print(get_distanceTraveled())
    elif oid == ".1.3.6.1.3.9999.9.0": # fuelLevel
        print(".1.3.6.1.3.9999.9.0")
        print("integer")
        print(get_fuelLevel())
    elif oid == ".1.3.6.1.3.9999.10.0": # batteryLevel
        print(".1.3.6.1.3.9999.10.0")
        print("integer")
        print(get_batteryLevel())
    elif oid == ".1.3.6.1.3.9999.11.0": # maintenanceMode
        print(".1.3.6.1.3.9999.11.0")
        print("integer")
        print(get_maintenanceMode())
    else:
        print("NONE1")

# Função para responder a requisições GETNEXT
def get_next_response(oid):
    next_oid = oid_map.get(oid, None)
    if next_oid:
        get_response(next_oid)
    else:
        print("NONE3")

# Função para responder a requisições SET
def set_response(oid, type, value):
    if oid == ".1.3.6.1.3.9999.5.0":  # desiredSpeed
        result = set_desiredSpeed(value)
        print(".1.3.6.1.3.9999.5.0")
        print(type)
        print(result)
    elif oid == ".1.3.6.1.3.9999.7.0":  # engineStatus
        result = set_engineStatus(value)
        print(".1.3.6.1.3.9999.7.0")
        print(type)
        print(result)
    elif oid == ".1.3.6.1.3.9999.11.0":  # maintenanceMode
        result = set_maintenanceMode(value)
        print(".1.3.6.1.3.9999.11.0")
        print(type)
        print(result)
    else:
        print(f"Unsupported OID: {oid}")

# Função principal para determinar o tipo de requisição
def main():
    load_values()  # Carrega os valores iniciais do arquivo, se existir

    if len(sys.argv) == 3:
        if sys.argv[1] == "-g":
            get_response(sys.argv[2])
        elif sys.argv[1] == "-gn":
            get_next_response(sys.argv[2])
    elif len(sys.argv) == 5:
        set_response(sys.argv[2], sys.argv[3], sys.argv[4])
    else:
        print("Usage: agent.py <MIB-oid> <request-type> <type> <value>")

    save_values()  # Salva os valores atualizados no arquivo

if __name__ == "__main__":
    main()


Overwriting /tmp/agent.py


## Restarting Agent

In [10]:
%%bash

sudo service snmpd restart
sudo service snmpd status

 * snmpd is running


## Testing GET

In [14]:
%%bash

snmpget -v2c -c public localhost .1.3.6.1.3.9999.1.0
snmpget -v2c -c public localhost .1.3.6.1.3.9999.2.0

snmpget -v2c -c public localhost .1.3.6.1.3.9999.3.0
snmpget -v2c -c public localhost .1.3.6.1.3.9999.4.0

snmpget -v2c -c public localhost .1.3.6.1.3.9999.5.0
snmpget -v2c -c public localhost .1.3.6.1.3.9999.6.0

snmpget -v2c -c public localhost .1.3.6.1.3.9999.7.0
snmpget -v2c -c public localhost .1.3.6.1.3.9999.8.0

snmpget -v2c -c public localhost .1.3.6.1.3.9999.9.0
snmpget -v2c -c public localhost .1.3.6.1.3.9999.10.0

snmpget -v2c -c public localhost .1.3.6.1.3.9999.11.0

SNMPv2-SMI::experimental.9999.1.0 = STRING: "Tesla"
SNMPv2-SMI::experimental.9999.2.0 = STRING: "S"
SNMPv2-SMI::experimental.9999.3.0 = STRING: "5YJ3E1EA7JF000000"
SNMPv2-SMI::experimental.9999.4.0 = STRING: "2023.20.11"
SNMPv2-SMI::experimental.9999.5.0 = INTEGER: 150


SNMPv2-SMI::experimental.9999.6.0 = INTEGER: 0
SNMPv2-SMI::experimental.9999.7.0 = INTEGER: 0
SNMPv2-SMI::experimental.9999.8.0 = INTEGER: 0
SNMPv2-SMI::experimental.9999.9.0 = INTEGER: 100
SNMPv2-SMI::experimental.9999.10.0 = INTEGER: 30
SNMPv2-SMI::experimental.9999.11.0 = INTEGER: 0


## Testing SET

In [13]:
%%bash

snmpset -v2c -c private localhost .1.3.6.1.3.9999.5.0 i 150

snmpset -v2c -c private localhost .1.3.6.1.3.9999.7.0 i 0

snmpset -v2c -c private localhost .1.3.6.1.3.9999.11.0 i 1

SNMPv2-SMI::experimental.9999.5.0 = INTEGER: 150


SNMPv2-SMI::experimental.9999.7.0 = INTEGER: 0
SNMPv2-SMI::experimental.9999.11.0 = INTEGER: 1


## Testing GETNEXT

In [31]:
%%bash

#snmpgetnext -v 1 -c public localhost ifDescr

snmpgetnext -v2c -c public localhost .1.3.6.1.3.9999.5.0

UCD-SNMP-MIB::memIndex.0 = INTEGER: 0


## Testing TABLE

In [20]:
%%bash

#snmptable -v 1 -c public localhost ifTable

snmptable -v2c -c public localhost #........

sensorEntry: Unknown Object Identifier (Sub-id not found: (top) -> sensorEntry)


CalledProcessError: Command 'b'\n#snmptable -v 1 -c public localhost ifTable\n\nsnmptable -v2c -c public localhost sensorEntry\n'' returned non-zero exit status 1.